In [1]:
%cd '/app'

/app


/home/user/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import os
import sys 
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
sys.path.append('/app/src')

In [4]:
from loader.data import _load_data, SimpleDataCollator, HybridDataCollator, BartDataCollator, BartPlusDataCollator, get_datacollator
from loader.model import load_model
from loader.checkpoint import load_pretrained_bag

In [5]:
task ='shape'
nvars = 2
field ='GF7'
encoding_method = 'hybrid'

density=0.3

data_name = f'{task}_n={nvars}_field={field}'
# data_name = f'{task}_n={nvars}_field={field}_density={density}'

data_path = f'data/{task}/{data_name}/data_{field}_n={nvars}'
data_config_path = f'config/{data_name}.yaml'

model = 'bart+'

group = f'{encoding_method}_{model}'
_save_path = f'{field}_n={nvars}'
# _save_path = f'{field}_n={nvars}_density={density}'
save_path = f'results/{task}/{group}/{_save_path}'

In [16]:
from evaluation.generation import generation_accuracy, generation, accuracy_score

In [7]:
from torch.utils.data import DataLoader
from misc.utils import to_cuda
import yaml, argparse
with open(os.path.join(save_path, 'params.yaml'), 'r') as f:
    params = yaml.safe_load(f)
    params = argparse.Namespace(**params)

bag = load_pretrained_bag(save_path, cuda=True)
model, tokenizer = bag['model'], bag['tokenizer']
model_name = params.model
print(f'Loaded model {model_name} from {save_path}')

data_encoding = 'infix+' if params.encoding_method == 'hybrid' and params.field == 'QQ' else 'infix'
testset     = _load_data(f'{params.data_path}.test.lex.{data_encoding}')
dc = get_datacollator(params.model)(tokenizer, continuous_coefficient=True)
# dc = get_datacollator(params.model)(tokenizer)
testloader = DataLoader(testset, batch_size=8, collate_fn=dc, shuffle=False)

Loaded model bart+ from results/shape/hybrid_bart+/GF7_n=2


In [8]:
from misc.utils import to_cuda
batch = next(iter(testloader))
batch = to_cuda(batch)
preds = generation(model, model_name, batch, tokenizer)

In [113]:
targets = tokenizer.batch_decode(batch['labels'], skip_special_tokens=True)

In [124]:
from evaluation.generation import support_eq, coeffs_eq

In [136]:
coeffs_eq(preds[0], batch['labels_for_regression'][0].detach().cpu().numpy(), th=0.4999)

True

In [178]:
accuracy_score(preds, targets, 
               labels_for_regression=batch['labels_for_regression'].detach().cpu().numpy(),
               th=0.4999)

{'acc': 0.5, 'support_acc': 0.75}

In [17]:
results = generation_accuracy(model_name,
                    model, 
                    testloader, 
                    batch_size=500, 
                    max_length=128,
                    tokenizer=tokenizer, 
                    th=0.4999, )

  0%|          | 0/125 [00:00<?, ?it/s]

100%|██████████| 125/125 [00:36<00:00,  3.40it/s]


In [25]:
results = generation_accuracy(params.model,
                    save_path, 
                    data_path + '.test.lex.infix', 
                    batch_size=500, 
                    max_length=128,
                    tokenizer=tokenizer, 
                    th=0.4999, )

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:27<00:00, 13.98s/it]


In [21]:
print(results.keys())
print(results['runtime_per_batch'])

dict_keys(['acc', 'support_acc', 'hits', 'support_hits', 'batch_runtimes', 'runtime_per_batch'])
0.28827040481567384


In [114]:
hit = preds[0] == targets[0]
support_hit = support_eq(preds[0], targets[0])

print(hit, support_hit)

False True


In [125]:
coeffs_eq(preds[0], batch['coeffs'][0], tokenizer)

KeyError: 'coeffs'

In [117]:
targets[0]

'[C] E1 E0 + [C] E0 E1 [SEP] [C] E0 E4 + [C] E0 E0'

In [118]:
preds[0]

'C1.0375462770462036 E1 E0 + C3.0773425102233887 E0 E1 [SEP] C1.0411913394927979 E0 E4 + C4.126354217529297 E0 E0'

In [31]:
dc = get_datacollator('bart+')

In [6]:
import yaml, argparse
with open(os.path.join(save_path, 'params.yaml'), 'r') as f:
    params = yaml.safe_load(f)
    params = argparse.Namespace(**params)

In [7]:
from loader.checkpoint import load_config, load_pretrained_bag, load_pretrained_model, load_tokenizer
model_config = load_config(save_path)
tokenizer = load_tokenizer(save_path)

In [14]:
bag = load_pretrained_bag(save_path, cuda=False)
model = bag['model']

In [13]:
# model = load_model(params.model, params, tokenizer=tokenizer, cuda=False)

In [10]:
# from transformers import BartForConditionalGeneration
# model = BartForConditionalGeneration.from_pretrained(os.path.join(save_path, f'model.safetensors'), config=model_config, use_safetensors=True)    

RuntimeError: Error(s) in loading state_dict for BartForConditionalGeneration:
	size mismatch for final_logits_bias: copying a param with shape torch.Size([1, 36]) from checkpoint, the shape in current model is torch.Size([1, 35]).
	size mismatch for lm_head.weight: copying a param with shape torch.Size([36, 512]) from checkpoint, the shape in current model is torch.Size([35, 512]).
	You may consider adding `ignore_mismatched_sizes=True` in the model `from_pretrained` method.

In [15]:
if params.encoding_method == 'hybrid' and params.field == 'QQ':
    data_encoding = 'infix+'
else:
    data_encoding = 'infix'

testset     = _load_data(f'{params.data_path}.test.lex.{data_encoding}')

In [16]:
use_continous_token = params.encoding_method == 'hybrid'    

if params.model == 'bart':
    dc = BartDataCollator(tokenizer)
    label_names = ['labels']

elif params.model == 'bart+':
    dc = BartPlusDataCollator(tokenizer, continuous_coefficient=use_continous_token)
    
    if use_continous_token:
        label_names = ['labels', 'labels_for_regression']
    else:
        label_names = ['labels']
else:
    raise ValueError(f'unknown model: {params.model}')


In [17]:
testloader = torch.utils.data.DataLoader(testset, batch_size=params.batch_size, shuffle=False, collate_fn=dc)

In [18]:
# model.cuda()

batch = next(iter(testloader))
# for k in batch:
#     if isinstance(batch[k], torch.Tensor):
#         batch[k] = batch[k].cuda()

In [19]:
for k in batch:
    print(k, batch[k].shape)

input_ids torch.Size([16, 273])
attention_mask torch.Size([16, 273])
decoder_input_ids torch.Size([16, 40])
decoder_attention_mask torch.Size([16, 40])
labels torch.Size([16, 40])
labels_for_regression torch.Size([16, 40, 1])
input_continuous_labels torch.Size([16, 273, 1])
target_continuous_labels torch.Size([16, 40, 1])


In [20]:
outputs = model(**batch)

In [77]:
outputs.logits.argmax(dim=-1)[0] == batch['labels'][0]

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True])

In [21]:
_batch = [sample for sample in testset][:10]
input_texts = [item["input"] for item in _batch]
target_texts = [item["target"] for item in _batch]
# input_encodings['attention_mask'].bool()

In [22]:
input_encodings = tokenizer(input_texts, padding='longest', return_tensors='pt')
target_encodings = tokenizer(target_texts, padding='longest', return_tensors='pt')

Exception: WordLevel error: Missing [UNK] token from the vocabulary

In [80]:
target_encodings['attention_mask']

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
  

NameError: name 'batch' is not defined

In [27]:
preds = model.generate(batch['input_ids'],  
                     #   num_beams=1, max_length=500, 
                     #   do_sample=False, 
                     batch['input_continuous_labels'],
                     continuous_token_ids=[tokenizer.vocab['[C]']],
                       attention_mask=batch['attention_mask'],) 
                     #   early_stopping=True)
preds_text = tokenizer.batch_decode(preds, skip_special_tokens=True)
labels_t = tokenizer.batch_decode(batch['labels'], skip_special_tokens=True)

IndexError: index out of range in self

In [86]:
batch['labels'].shape

torch.Size([16, 96])

In [85]:
no_pad = batch['labels'] != tokenizer.pad_token_id
hits = preds[:, 1:][no_pad] == batch['labels'][no_pad]
hits.float().mean()

IndexError: The shape of the mask [16, 96] at index 1 does not match the shape of the indexed tensor [16, 108] at index 1

In [64]:
batch['labels'][0]

tensor([ 6, 14, 13, 13, 13, 34,  8, 13, 13, 13, 16, 34,  9, 13, 13, 13, 15, 42,
         6, 13, 14, 13, 13, 34,  7, 13, 13, 13, 16, 34,  6, 13, 13, 13, 15, 34,
         9, 13, 13, 13, 14, 42,  6, 13, 13, 14, 13, 34,  7, 13, 13, 13, 16, 34,
        10, 13, 13, 13, 13, 42,  6, 13, 13, 13, 17, 34,  9, 13, 13, 13, 14, 40,
        38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38,
        38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38,
        38, 38, 38, 38, 38, 38])

In [87]:
input_text = tokenizer.batch_decode(batch['input_ids'], skip_special_tokens=True)
preds_text = tokenizer.batch_decode(preds, skip_special_tokens=True)
labels_t = tokenizer.batch_decode(batch['labels'], skip_special_tokens=True)

In [88]:
hits = []
for pt, lt in zip(preds_text, labels_t):
    hits.append(pt == lt)
    if pt == lt:
        print(f'[o] pred: {pt}\n   label: {lt}\n\n')
    else:
        print(f'[x] pred: {pt}\n   label: {lt}\n\n')

[o] pred: C1 E1 E0 E0 E0 + C8 E0 E0 E0 E0 [SEP] C1 E0 E1 E0 E0 + C5 E0 E0 E0 E0 [SEP] C1 E0 E0 E1 E0 + C7 E0 E0 E0 E0 [SEP] C1 E0 E0 E0 E1
   label: C1 E1 E0 E0 E0 + C8 E0 E0 E0 E0 [SEP] C1 E0 E1 E0 E0 + C5 E0 E0 E0 E0 [SEP] C1 E0 E0 E1 E0 + C7 E0 E0 E0 E0 [SEP] C1 E0 E0 E0 E1


[x] pred: C1 E1 E0 E0 E0 + C11 E0 E0 E0 E4 + C27 E0 E0 E0 E3 + C23 E0 E0 E0 E2 [SEP] C1 E0 E1 E0 E0 + C24 E0 E0 E0 E0 [SEP] C1 E0 E0 E1 E0 + C2 E0 E0 E0 E0 [SEP] C1 E0 E0 E0 E5 + C5 E0 E0 E0 E4 + C20 E0 E0 E0 E2
   label: C1 E1 E0 E0 E0 + C11 E0 E0 E0 E4 + C27 E0 E0 E0 E3 + C23 E0 E0 E0 E2 [SEP] C1 E0 E1 E0 E0 + C24 E0 E0 E0 E0 [SEP] C1 E0 E0 E1 E0 + C2 E0 E0 E0 E0 [SEP] C1 E0 E0 E0 E5 + C5 E0 E0 E0 E4 + C25 E0 E0 E0 E2


[o] pred: C1 E1 E0 E0 E0 + C29 E0 E0 E0 E1 + C2 E0 E0 E0 E0 [SEP] C1 E0 E1 E0 E0 + C26 E0 E0 E0 E4 + C11 E0 E0 E0 E3 + C6 E0 E0 E0 E1 + C7 E0 E0 E0 E0 [SEP] C1 E0 E0 E1 E0 + C11 E0 E0 E0 E4 + C2 E0 E0 E0 E3 + C2 E0 E0 E0 E2 + C15 E0 E0 E0 E1 [SEP] C1 E0 E0 E0 E5 + C22 E0 E0 E0 E4 + C3 E0 E0 E0

In [89]:
acc = sum(hits) / len(hits)
acc

0.5625

In [116]:
input_text[-1]

'C4 E2 E0 + C7 E1 E7 + C4 E1 E6 + C2 E1 E5 + C8 E1 E4 + C30 E1 E3 + C28 E1 E2 + C20 E1 E1 + C9 E1 E0 [SEP] C6 E4 E1 + C26 E3 E8 + C6 E3 E7 + C3 E3 E6 + C12 E3 E5 + C14 E3 E4 + C11 E3 E3 + C30 E3 E2 + C14 E3 E1 + C22 E2 E2 + C5 E2 E1 + C13 E1 E9 + C3 E1 E8 + C17 E1 E7 + C6 E1 E6 + C7 E1 E5 + C21 E1 E4 + C15 E1 E3 + C3 E1 E2 + C1 E0 E5 + C5 E0 E4 + C22 E0 E3 + C30 E0 E2 + C17 E0 E1 [SEP] C16 E4 E3 + C19 E3 E4 + C5 E3 E3 + C24 E3 E2 + C11 E2 E9 + C24 E2 E8 + C12 E2 E7 + C17 E2 E6 + C25 E2 E5 + C17 E2 E4 + C27 E2 E3 + C23 E2 E2 + C10 E2 E0 + C3 E1 E7 + C15 E1 E6 + C27 E1 E5 + C19 E1 E4 + C30 E1 E3 + C8 E1 E2 + C19 E1 E1 + C8 E1 E0 + C10 E0 E0'

In [117]:
labels_t[-1]

'C1 E1 E0 + C10 E0 E0 [SEP] C1 E0 E5 + C5 E0 E4 + C22 E0 E3 + C30 E0 E2 + C17 E0 E1'

In [5]:
from dataset.tokernizer import set_tokenizer, set_vocab

In [16]:
num_variables = 2
field = 'QQ'
max_coefficient = 100
max_degree=20
continous_ids = [2]

params = {'encoding_method': 'standard',
          'd_model': 256,
          'nhead': 4,
          'num_encoder_layers': 2,
          'num_decoder_layers': 2,
          'dim_feedforward': 1024,
          'dropout': 0.1,
          'max_sequence_length': 512,
          'positional_encoding': 'embedding',
          'regression_weight': 1.0,
          'continuous_coefficient': False, # 
          'continuous_exponent': False, #
          'regression_weights': [1.0], #
          'continuous_embedding_model': 'ffn'} #

import argparse

params = argparse.Namespace(**params)

vocab = set_vocab(num_variables, 
                  field=field, 
                  max_coeff=max_coefficient, 
                  max_degree=max_degree, 
                  continuous_coefficient=False, 
                continuous_exponent=False)
tokenizer = set_tokenizer(vocab)
model = load_model(params, 'bart', vocab=vocab, tokenizer=tokenizer)


In [8]:
task = 'shape'
encoding = 'hybrid'
n, field = 2, 'GF7'
data_name = f'{task}_n={n}_field={field}'
data_path = f'data/{task}/{data_name}/data_{field}_n={n}.test.lex.infix'

testset     = _load_data(data_path)

In [9]:
dc = HybridDataCollator(tokenizer)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, collate_fn=dc, shuffle=False)

In [10]:
batch = next(iter(testloader))

In [46]:
# task = 'shape'
# encoding = 'standard'

# n, field = 2, 'GF7'
# data_name = f'{task}_n={n}_field={field}'
# data_path = f'data/{task}/{data_name}/data_{field}_n={n}.test.lex.infix'

# data_config_path = f'config/{data_name}.yaml'
# _save_path = f'{field}_n={n}_ep=8_bs=16'
# save_path = f'results/{task}/{encoding}_embedding/{_save_path}'
# # save_path = f'results/{task}/{encoding}/dryrun'

In [11]:
for key in batch:
    batch[key] = batch[key].to(model.device) if isinstance(batch[key], torch.Tensor) else batch[key]

In [12]:
model(**batch)

{'loss': tensor(19.9327, device='cuda:0', grad_fn=<AddBackward0>),
 'loss_clf': tensor(5.6199, device='cuda:0', grad_fn=<NllLossBackward0>),
 'loss_rg': tensor(14.3128, device='cuda:0', grad_fn=<MseLossBackward0>),
 'logits': tensor([[[-0.7997,  0.1777, -0.4062,  ...,  0.0946, -0.1072, -0.6497],
          [ 0.1215,  0.7225, -0.7747,  ...,  1.2960, -0.0859, -0.2309],
          [-0.5851, -0.0074, -0.6108,  ..., -0.1392,  0.0496, -1.0368],
          ...,
          [-0.2318,  0.3845, -1.1978,  ...,  0.1619, -0.4243,  1.0051],
          [-0.5707,  0.4581, -0.5036,  ...,  0.0650, -0.1025,  0.2354],
          [-0.5423, -0.5598, -0.8385,  ...,  0.3655,  1.3766, -0.0080]],
 
         [[-0.8181, -0.0689, -0.1057,  ..., -0.0082, -0.4242, -0.6536],
          [ 0.0495,  0.5423, -0.7644,  ...,  1.3736, -0.0865, -0.0540],
          [-0.6605,  0.0864, -0.4809,  ..., -0.0854,  0.1100, -1.1429],
          ...,
          [-0.3453,  0.0089, -1.0978,  ...,  0.1233, -0.3788,  0.5984],
          [-0.4874,  0

In [32]:
model.greedy_generate(batch['encoder_input'], 
                      encoder_input_labels=batch['encoder_input_labels'], 
                      max_length=500, 
                      encoder_padding_mask=batch['encoder_padding_mask'], 
                      continuous_token_ids=torch.tensor([tokenizer.vocab['[C]']]).to(model.device),
                      )

(tensor([[231, 136, 136,  ...,  26,  25,  10],
         [231,  54, 149,  ..., 170, 151,  89],
         [231,  54, 149,  ...,  10, 173,   5],
         [231,  54, 186,  ..., 172,  79,  91]], device='cuda:0'),
 tensor([[inf, inf, inf,  ..., inf, inf, inf],
         [inf, inf, inf,  ..., inf, inf, inf],
         [inf, inf, inf,  ..., inf, inf, inf],
         [inf, inf, inf,  ..., inf, inf, inf]], device='cuda:0'))

In [33]:
num_variables = 2
field = 'QQ'
max_coefficient = 100
max_degree=20
continous_ids = [2]

params = {'encoding_method': 'standard',
          'd_model': 256,
          'nhead': 4,
          'num_encoder_layers': 2,
          'num_decoder_layers': 2,
          'dim_feedforward': 1024,
          'dropout': 0.1,
          'max_sequence_length': 512,
          'positional_encoding': 'embedding',
          'regression_weight': 1.0,}

import argparse

params = argparse.Namespace(**params)

vocab = set_vocab(num_variables, 
                  field=field, 
                  max_coeff=max_coefficient, 
                  max_degree=max_degree, 
                  continuous_coefficient=False, 
                continuous_exponent=False)
tokenizer = set_tokenizer(vocab)
model = load_model(params, vocab=vocab, tokenizer=tokenizer)


In [124]:
bag = load_pretrained_bag(save_path)
config, model, tokenizer = bag['config'], bag['model'], bag['tokenizer']

NameError: name 'load_pretrained_bag' is not defined

In [156]:
from loader.data import _load_data
from loader.data import SimpleDataCollator
from torch.utils.data import DataLoader
import numpy as np 
from tqdm import tqdm

batch_size = 500
dataloader = data_path
disable_tqdm = False
max_length = 10000

if isinstance(dataloader, str):
    dataset = _load_data(dataloader)
    dc = SimpleDataCollator(tokenizer)
    dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=dc, shuffle=False)

# load model    
if isinstance(model, str):
    bag = load_pretrained_bag(model)
    config, model, tokenizer = bag['config'], bag['model'], bag['tokenizer']
else:
    assert(tokenizer is not None)


model.cuda()

with torch.no_grad():
    hits = []
    dataloader = tqdm(dataloader, disable=disable_tqdm)  if not disable_tqdm else dataloader
    for batch in dataloader:
        max_length = min(max_length, batch['labels'].shape[1] + 1)
        outputs = model.greedy_generate(batch['encoder_input'].cuda(), max_length=max_length, encoder_padding_mask=batch['encoder_padding_mask'].cuda())
        pred = tokenizer.batch_decode(outputs.cpu().numpy(), skip_special_tokens=True)
        target = tokenizer.batch_decode(batch['labels'], skip_special_tokens=True)
        
        hits += [p == t for p, t in zip(pred, target)]
        
    ret = {'acc': np.array(hits, dtype=float).mean(), 'hits': hits}

100%|██████████| 2/2 [00:10<00:00,  5.01s/it]


In [157]:
ret['acc']

0.432

: 

In [132]:
ret['hits'][:10]

[True, False, True, True, True, False, True, False, True, False]

In [133]:
batch.keys()

dict_keys(['encoder_input', 'decoder_input', 'encoder_padding_mask', 'decoder_padding_mask', 'labels'])

In [134]:
batch  = next(iter(dataloader))

In [149]:
%%time
outputs = model.greedy_generate(batch['encoder_input'].cuda(), 
                                encoder_attention_mask=None,
                                encoder_padding_mask=batch['encoder_padding_mask'].cuda(),
                                max_length=max_length)

CPU times: user 20.9 s, sys: 48.1 ms, total: 20.9 s
Wall time: 4.61 s


In [150]:
outputs[:4]

tensor([[36,  4, 12, 11, 32,  4, 11, 11, 40,  4, 11, 12, 38, 36, 36, 36, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36, 36],
        [36,  4, 12, 11, 32,  4, 11, 13, 40,  4, 11, 14, 32,  5, 11, 13, 32,  5,
         11, 12, 32,  5, 11, 11, 38, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36, 36],
        [36,  4, 12, 11, 32,  5, 11, 11, 40,  4, 11, 15, 38, 36, 36, 36, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36, 36],
        [36,  4, 12, 11, 32,  8, 11, 14, 32,  9, 11, 12, 32,  9, 11, 11, 40,  4,
         11, 15, 32,  9, 11, 14, 32,  8, 11, 13, 32,  8, 11, 12, 38, 36, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36, 36]], device='cuda:0')

In [142]:
batch['labels'][:4]

tensor([[ 4, 12, 11, 32,  4, 11, 11, 40,  4, 11, 12, 38, 36, 36, 36, 36, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36],
        [ 4, 12, 11, 32,  4, 11, 13, 40,  4, 11, 14, 32,  8, 11, 11, 38, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36],
        [ 4, 12, 11, 32,  5, 11, 11, 40,  4, 11, 15, 38, 36, 36, 36, 36, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36],
        [ 4, 12, 11, 32,  8, 11, 14, 32,  9, 11, 12, 32,  9, 11, 11, 40,  4, 11,
         15, 32,  9, 11, 14, 32,  8, 11, 13, 32,  8, 11, 12, 38, 36, 36, 36, 36,
         36, 36, 36, 36, 36, 36, 36, 36]])

In [151]:
tokenizer.batch_decode(outputs.cpu().numpy(), skip_special_tokens=True)[1]

'C1 E1 E0 + C1 E0 E2 [SEP] C1 E0 E3 + C2 E0 E2 + C2 E0 E1 + C2 E0 E0'

In [147]:
tokenizer.batch_decode(batch['labels'], skip_special_tokens=True)[1]

'C1 E1 E0 + C1 E0 E2 [SEP] C1 E0 E3 + C5 E0 E0'

In [148]:
tokenizer.batch_decode(batch['encoder_input'], skip_special_tokens=True)[1]


'C1 E1 E0 + C1 E0 E2 [SEP] C1 E3 E1 + C1 E2 E3 + C5 E2 E2 + C5 E1 E4 + C1 E1 E2 + C5 E1 E1 + C3 E1 E0 + C6 E0 E3 + C3 E0 E2 + C2 E0 E1 + C5 E0 E0 [SEP] C4 E5 E2 + C4 E4 E4 + C4 E4 E1 + C4 E3 E3 + C6 E3 E2 + C5 E3 E1 + C3 E2 E5 + C3 E2 E4 + C5 E2 E3 + C3 E2 E1 + C1 E2 E0 + C2 E1 E4 + C1 E1 E3 + C1 E1 E2 + C2 E0 E4 + C5 E0 E3 + C1 E0 E2 + C3 E0 E1 + C2 E0 E0'

In [71]:
tokenizer.vocab

{'E9': 20,
 'C0': 3,
 'E12': 23,
 '[UNK]': 41,
 '/': 35,
 'E10': 21,
 'E2': 13,
 'C2': 5,
 'E17': 28,
 '[SEP]': 40,
 '^': 34,
 'E14': 25,
 'E18': 29,
 '</s>': 38,
 '<s>': 37,
 'E3': 14,
 '+': 32,
 'C3': 6,
 'E8': 19,
 'E13': 24,
 'E0': 11,
 'E5': 16,
 '[PAD]': 36,
 'E4': 15,
 'C4': 7,
 'E20': 31,
 'C1': 4,
 'C5': 8,
 '[C]': 2,
 'C6': 9,
 '[E]': 10,
 'E19': 30,
 '[CLS]': 39,
 '*': 33,
 'E16': 27,
 'E1': 12,
 'E11': 22,
 'x0': 0,
 'x1': 1,
 'E15': 26,
 'E7': 18,
 'E6': 17}